In [14]:
#Using python SPAKE2 Library (ed25519 curve)
from spake2_local.spake2 import SPAKE2_A
from spake2_local.spake2 import SPAKE2_B
from hkdf import Hkdf
import hmac
from hashlib import sha256

# generate SPAKE instance , PAKE message and shared key

In [15]:
# input register code
shared_password=b"123456" #byte

# use regiser code to generate SPAKE instance 
mobile = SPAKE2_A(shared_password) #1. 공동의 비밀번호를 Scalar로 변환 (__init__)
car = SPAKE2_B(shared_password)
print(mobile.pw_scalar)
print(car.pw_scalar)


2068500508171551459763435228377381679810713647176111590801746788672287807293
2068500508171551459763435228377381679810713647176111590801746788672287807293


In [16]:
#Generate PAKE message 
mobile_spake_message = mobile.start()
car_spake_message = car.start()
print(f"mobile_pake_message : {mobile_spake_message} / size : {len(mobile_spake_message)}")
print(f"car_pake_message : {car_spake_message} / size : {len(car_spake_message)}")

mobile_pake_message : b'm\xa4\xa6/#N\xaf\x86\x14\xd8\xd9[\xd4\x9b\xc5?\x9d\xc4o\x901.Tu\xafg\xaf\xa7ju8s\xa0' / size : 33
car_pake_message : b'v/u\x08<^#\x82\x8cq\xf6\xbe\xc8\xc9,\xc0=\xc3!\xaf\xbe\x0c\xab\x8e\xa1%\xf3\xf2\xeb\xe6\xaa\xac\xf4' / size : 33


In [17]:
#Generate Shared Key = xyG
mobile_shared_value = mobile.finish(car_spake_message)
car_shared_value = car.finish(mobile_spake_message)

print(mobile_shared_value)
print(car_shared_value)

b'&\xaefL%\x90\xb2\x82\xc4\x80\x17}\x8c\x921w\x9c\x8ab\x95\xc5\xdf\x03\xb1\x85\x15J\x92\xe9\xcaz\xa6'
b'&\xaefL%\x90\xb2\x82\xc4\x80\x17}\x8c\x921w\x9c\x8ab\x95\xc5\xdf\x03\xb1\x85\x15J\x92\xe9\xcaz\xa6'


# generate shared key and confirmation key

In [23]:
def length_to_bytes(length_in_hex):
    length = length_in_hex[2:]
    if len(length)<2:
        length = "0"+length
    return length+"00"*7 #7byte padding 
#generate confirmation key 

mobile_name_length = length_to_bytes(hex(len(mobile.side)))
car_name_length = length_to_bytes(hex(len(car.side)))
mobile_spake_message_length = length_to_bytes(hex(len(mobile_spake_message)))
car_spake_message_length = length_to_bytes(hex(len(car_spake_message)))
shared_value_length = length_to_bytes(hex(len(mobile_shared_value)))
shared_password_length = length_to_bytes(hex(len(hex(mobile.pw_scalar))))

TT = "".join([
    mobile_name_length, # 모바일 측 이름 (Client) 길이 
    mobile.side.hex(), # 모바일 측 이름 (Client)의 hex
    car_name_length, # 자동차 측 이름 (server) 길이
    car.side.hex(), # 자동차 측 이름 server hex
    mobile_spake_message_length, # mobile 에서 보낸 spake 메시지
    mobile_spake_message.hex(), 
    car_spake_message_length, # car spake message 
    car_spake_message.hex(),
    shared_value_length, # spake 를 통해 만들어진 공유 키 길이 (양측 동일 )
    mobile_shared_value.hex(), # spake 를 통해 만들어진 공유 키 hex (양측 동일 )
    shared_password_length, # 공동 패스워드(register code)로 만든 scalar 의 길이 
    hex(mobile.pw_scalar)[2:] # 공동 패스워드(register code)로 만든 scalar 의 hex
])

hashed_TT = sha256(TT.encode()).hexdigest()
Ka = hashed_TT[:int(len(hashed_TT)/2)]
Ke = hashed_TT[int(len(hashed_TT)/2):]
# both parties can make TT thus, Ka, Ke
print(hashed_TT)
print(Ka, "<-- only used for key confirmation")
print(Ke, "<-- Shared Key ") 

2deb58c863e58e34ec9a4c2f08bf2c53e963592b188c064780d3be6fd4d57624
2deb58c863e58e34ec9a4c2f08bf2c53 <-- only used for key confirmation
e963592b188c064780d3be6fd4d57624 <-- Shared Key 


In [25]:
# Generate confrimation key 

hk = Hkdf(salt=bytes.fromhex(''),input_key_material=bytes.fromhex(b"ConfirmationKeys".hex())) 
# HKDF 생성 -- salt 값은 공백, input 값은 "ConfirmationKeys" 문자열의 hex byte 값 
okm = hk.expand(info=bytes.fromhex(Ka)) # Key confirmaiton을 위한 Ka의 hex byte를 HKDF에 인풋으로 입력한 결과 값 
Kc = sha256(okm).hexdigest() #결과 값의 sha256 해시

KcA = Kc[:int(len(Kc)/2)] 
ConfirmationKey_A = hmac.new(KcA.encode(),TT.encode(),sha256).hexdigest() #해시의 앞 절반은 모바일 --> 자동차 

KcB = Kc[int(len(Kc)/2):] 
ConfirmationKey_B = hmac.new(KcB.encode(),TT.encode(),sha256).hexdigest() #해시의 뒤 절반은 자동차 --> 모바일 

In [26]:
print(f"Kc : {Kc}")
print(f"KcA : {KcA}")
print(f"ConfirmationKey_A : {ConfirmationKey_A}")

print(f"KcB : {KcB}")
print(f"ConfirmationKey_B : {ConfirmationKey_B}")


Kc : e47b88c867e2034cb1d93599b6c16fba161fca20971c4e819c94667c295276d0
KcA : e47b88c867e2034cb1d93599b6c16fba
ConfirmationKey_A : 3f9b9b8de2e4d6cfd55f02814492cd0606da97d033a04e439f600889603711e2
KcB : 161fca20971c4e819c94667c295276d0
ConfirmationKey_B : 6f39e92359c3c61b4501e1d5b1306dd1ffbdaf1920e2515b53a048bb9a3c6f5b


In [27]:
# 모바일 측에서는 자동차에서 보낸 ConfirmationKey_B를 받아서 검증
mobile.key_confirmation(TT,ConfirmationKey_B)

True

In [28]:
# 자동차 측에서는 모바일에서 보낸 ConfirmationKey_B를 받아서 검증
car.key_confirmation(TT,ConfirmationKey_A)

True

# Challenge process

In [29]:
from Crypto import Random
from Crypto.Cipher import AES

BLOCK_SIZE = 16

def pad(s):
    s = s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * chr(BLOCK_SIZE - len(s) % BLOCK_SIZE).encode()
    return s

def unpad(encrypted_string):
    return encrypted_string[0:-(encrypted_string[-1])]


class AESCipher(object):
    def __init__(self, key):
        self.key = key.encode()

    def encrypt(self, raw):
        raw = pad(raw)
        iv = Random.new().read(AES.block_size)
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        # print(f"cipher len : {len(cipher.encrypt(raw))}")
        return iv + cipher.encrypt(raw)
    
    def decrypt(self, enc):
        iv = enc[:BLOCK_SIZE]
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return unpad(cipher.decrypt(enc[BLOCK_SIZE:]))

def AES_key_padding(z):
    if len(z) > 32 : 
        z = z[:32]
    else:
        for i in range(32-len(z)):
            z = z+"0"
    return z 

In [30]:
#Generate Nonce 
import binascii
import os 
import struct 
import datetime

nonce_c = int(binascii.hexlify(os.urandom(4)),16).to_bytes(4,"big")# 4byte random int
ts = struct.pack(">i",int(datetime.datetime.timestamp(datetime.datetime.now())))
challenge = nonce_c+ts

### Car --> Phone Challenge

In [31]:
#AES key는 16의 배수이므로 키의 길이를 32바이트로 만들기 위한 패딩작업 
AES_key1 = AES_key_padding(shared_password.decode())
AES_key2 = AES_key_padding(Ke)

Car_cipher1 = AESCipher(key=AES_key1) # 2중 암호화를 위해 Shared password를 Key로 암호화 인스턴스1 생성 
Car_cipher2 = AESCipher(key=AES_key2)  # 2중 암호화를 위해 Ke를 Key로 암호화 인스턴스2 생성 

# todo 혼다측 요구사항 (2중암호화)을 위해 register code로 한번 더 암호화 해야됨 
# shared_password로 1차 암호화 한 후 
# spake를 통해 만들어진 Ke로 2차 암호화 

plain_challenge_from_car = challenge
print("plain    :",plain_challenge_from_car)

encrypted_challenge_from_car1 = Car_cipher1.encrypt(plain_challenge_from_car)
print("encrypted1:",encrypted_challenge_from_car1)

encrypted_challenge_from_car2 = Car_cipher2.encrypt(encrypted_challenge_from_car1)
print("encrypted2:",encrypted_challenge_from_car2)


plain    : b'\xbf+D\xdccu\x82%'
encrypted1: b'H\x10\xd4\x05\xff\xd1\xabd\xb9~Y:\xc9j\xd6\xad\xc7w\xc3\xe0\x04\x8b\xdd\xecG2[I\x03\x0b\x04\xd6'
encrypted2: b'\xba\x02!]\xf4\x85o\x14J\xe8@\x90\xb6e+\xb5\xac\xc6<\x8d\x98\xeamf*\x10s\x90\xb0:\xf9\x9e\x14\xab6\xf8X&\xe7\xde\xa7\x08\xcd6\xb6\xaf\x88\x1f\xd0\x96L\x1cg\xed\x84\xca`?\x97\t\x1bA\x02\x91'


### Phone --> Car Challenge

In [32]:
#AES key는 16의 배수이므로 키의 길이를 32바이트로 만들기 위한 패딩작업 
AES_key1 = AES_key_padding(shared_password.decode())
AES_key2 = AES_key_padding(Ke)

Phone_cipher1 = AESCipher(key=AES_key1)  # 2중 암호화를 위해 Shared password를 Key로 암호화 인스턴스1 생성 
Phone_cipher2 = AESCipher(key=AES_key2)  #  2중 암호화를 위해 Ke를 Key로 암호화 인스턴스2 생성 

# 2번 복호화 
decrypted_challenge_from_car1 = Phone_cipher2.decrypt(encrypted_challenge_from_car2)
decrypted_challenge_from_car = Phone_cipher1.decrypt(decrypted_challenge_from_car1)

print("decrypted:",decrypted_challenge_from_car)

decrypted: b'\xbf+D\xdccu\x82%'


In [33]:
username = "AUTOCRYPT"
padded_username = int(username.encode().hex(),16).to_bytes(64,"big")

In [34]:
plain_response_from_phone = decrypted_challenge_from_car+padded_username


In [35]:
# 2중암호화 
encrypted_response_from_phone1 = Phone_cipher1.encrypt(plain_response_from_phone)
encrypted_response_from_phone2 = Phone_cipher2.encrypt(encrypted_response_from_phone1)

### Car respose check (challenge solve check)

In [36]:
#AES key는 16의 배수이므로 키의 길이를 32바이트로 만들기 위한 패딩작업 
AES_key1 = AES_key_padding(shared_password.decode())
AES_key2 = AES_key_padding(Ke)

Car_cipher1 = AESCipher(key=AES_key1) # 2중 암호화를 위해 Shared password를 Key로 암호화 인스턴스1 생성 
Car_cipher2 = AESCipher(key=AES_key2)  # 2중 암호화를 위해 Ke를 Key로 암호화 인스턴스2 생성 

#2중 복호화 
decrypted_response_from_phone1 = Car_cipher2.decrypt(encrypted_response_from_phone2)
decrypted_response_from_phone = Car_cipher1.decrypt(decrypted_response_from_phone1)

In [37]:
responsed_nonce_c = decrypted_response_from_phone[0:4]
responsed_timestamp_c = decrypted_response_from_phone[4:8]
responsed_username = decrypted_response_from_phone[8:]

In [38]:
responsed_nonce_c == nonce_c

True

In [39]:
nowtime = int(datetime.datetime.timestamp(datetime.datetime.now()))
challenge_time = int(responsed_timestamp_c.hex(),16)
elapsed_minute = (nowtime-challenge_time)/60 

In [40]:
elapsed_minute

0.15

In [41]:
# 2분 이내에 challenge 해결해야할 경우 
elapsed_minute < 2 

True

### Save Shared Key (Ke) to safe location


In [42]:
# RPS 시스템은 안전한 저장을 위해 NXP 칩셋 분석 필요 
safe_file = open("./key","w")
safe_file.write(Ke)
safe_file.close()